In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00


In [17]:
import logging
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from sklearn.metrics import accuracy_score, f1_score
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ls /content/drive/MyDrive/Ind_Proj/large_language_models

LLama_2_7b_chat_multidataset_finetuned	Sentences_AllAgree.txt


In [6]:
!cp -r /content/drive/MyDrive/Ind_Proj/large_language_models/Sentences_AllAgree.txt /content/

In [7]:
!ls /content/

drive  sample_data  Sentences_AllAgree.txt


In [8]:
# Define a custom loading function
def process_line(line):
    # Split the line by the @ symbol
    text, label = line.rsplit('@', 1)
    return {"text": text.strip(), "label": label.strip()}

# Load the dataset from your text file
dataset = load_dataset('text', data_files='Sentences_AllAgree.txt',split='train', encoding='ISO-8859-1')

# Map the text file lines to 'text' and 'label' features
dataset = dataset.map(lambda example: process_line(example['text']))

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2264 [00:00<?, ? examples/s]

In [27]:
# Shuffle the dataset and select the first 1000 rows
fpb_dataset = dataset.shuffle(seed=42).select(range(1000))

# Check the new dataset
print(fpb_dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})


In [10]:
!ls /content/drive/MyDrive/Ind_Proj/large_language_models

LLama_2_7b_chat_multidataset_finetuned	Sentences_AllAgree.txt


In [11]:
!cp -r /content/drive/MyDrive/Ind_Proj/large_language_models/LLama_2_7b_chat_multidataset_finetuned /content/Llama-2-7b-chat-finetuned

In [12]:
!ls /content/Llama-2-7b-chat-finetuned

adapter_config.json  adapter_model.bin	README.md


In [13]:
model_name = "NousResearch/Llama-2-7b-chat-hf"

In [14]:
new_model = "Llama-2-7b-chat-finetuned"

In [15]:
# Load the entire model on the GPU 0
device_map = {"": 0}

In [18]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/peft_model.py:556: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapters_weights = torch.load(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [19]:
# Define the text-generation pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

In [20]:
# Function to generate sentiment
def generate_sentiment(text):
    # Construct the prompt
    prompt = f"Determine the sentiment of the financial news as negative, neutral or positive: {text}"

    # Run the text generation
    result = pipe(f"<s>[INST] {prompt} [/INST]")

    # Extract the generated text
    generated_text = result[0]['generated_text']

    # Extract the text after [/INST]
    return extract_after_inst(generated_text)

In [21]:
# Function to extract the part after [/INST]
def extract_after_inst(generated_text):
    parts = generated_text.split('[/INST]')
    return parts[1].strip() if len(parts) > 1 else generated_text

In [22]:
# Function to check sentiment in the model's output
def check_sentiment(output):
    output = output.lower()
    if "negative" in output:
        return "negative"
    elif "neutral" in output:
        return "neutral"
    elif "positive" in output:
        return "positive"
    else:
        return "neutral"  # Default to "neutral" if no keyword found

In [28]:
def generate_and_classify_batch(batch):
    texts = batch['text']
    actual_labels = batch['label']

    # Generate predictions for the batch
    generated_texts = pipe([f"<s>[INST] Determine the sentiment of the financial news as negative, neutral or positive: {text} [/INST]" for text in texts])

    # Flattening the nested lists of generated texts
    if isinstance(generated_texts[0], list):
        generated_texts = [item for sublist in generated_texts for item in sublist]

    # Extract and classify sentiment from generated texts
    predicted_sentiments = [check_sentiment(extract_after_inst(result['generated_text'])) for result in generated_texts]

    print("Batch completed.")

    return {'y_true': actual_labels, 'y_pred': predicted_sentiments}

# Apply the transformation using map with batch_size
batch_size = 64
result = fpb_dataset.map(generate_and_classify_batch, batched=True, batch_size=batch_size)

# Extract true and predicted labels for evaluation
y_true = result['y_true']
y_pred = result['y_pred']

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.


In [29]:
from sklearn.metrics import accuracy_score, f1_score

# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

In [30]:
# Evaluate the model using accuracy and F1 score
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted')

# Print the evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.9130
F1 Score: 0.9140


In [32]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Determine the sentiment of the financial news as negative, neutral or positive:Consumer credit $18.9BN, Exp. $16BN, Last $9.6BN."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=50)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Determine the sentiment of the financial news as negative, neutral or positive:Consumer credit $18.9BN, Exp. $16BN, Last $9.6BN. [/INST] positive
